In [ ]:
<a href="https://colab.research.google.com/github/rgclapp007/gp211-class-notebooks/blob/main/regularization/galilee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install "sep_plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@2bffacb9fb36963339a0834c2b04a0aedff91db4"

In [ ]:
!wget https://raw.githubusercontent.com/rgclapp007/gp211-class-notebooks/main/data/galilee.H

In [ ]:
import sep_python.modes
import numpy as np
from sep_python.hypercube import Hypercube
from sep_python.sep_vector import FloatVector
io=sep_python.modes.default_io
vec=io.vector_from_storage("./galilee.H")




In [ ]:

from genericSolver.pyOperator import Operator
import numba

@numba.njit(parallel=True)
def find_outside(n1,n2,ip1,ip2,f1,f2,e1,e2):
  for i in numba.prange(ip1.shape[0]):
    if ip1[i] < 0 or ip2[i] <0 or ip1[i] >=n1-1 or\
      ip2[i] <0 or ip2[i] >=n2-1:
      f1[i]=0
      f2[i]=0
      e1[i]=0
      e2[i]=0
      ip1[i]=0
      ip2[i]=0



class linear_interp_2d(Operator):
  def __init__(self,model:FloatVector,data:FloatVector,x,y):
    if not isinstance(model,FloatVector) or not isinstance(data,FloatVector):
      raise Exception("wrong input")
    axes=model.get_hyper().axes
    if len(axes)!=2:
      raise Exception("expecting model to be 2-D")

    self._f1=(x-axes[0].o)/axes[0].d
    self._f2=(y-axes[1].o)/axes[1].d
    self._ipos2=np.int_(self._f2)
    self._ipos1=np.int_(self._f1)
    print("range i see ",self._ipos1.min(),self._ipos1.max(),self._ipos2.min(),self._ipos2.max())
    self._f1-=self._ipos1
    self._f2-=self._ipos2
    self._e1=1.-self._f1
    self._e2=1.-self._f2
    find_outside(axes[0].n,axes[1].n,self._ipos1,self._ipos2,self._f1,\
        self._f2,self._e1,self._e2)
    super().__init__(model,data)

  def forward(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      data.zero()
    d=data.get_nd_array()
    m=model.get_nd_array()
    forward_it(m,d,self._ipos1,self._ipos2,self._f1,self._f2,self._e1,self._e2)
    
  def adjoint(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      model.zero()
    m=model.get_nd_array()
    d=data.get_nd_array()
    adjoint_it(m,d,self._ipos1,self._ipos2,self._f1,self._f2,self._e1,self._e2)

  

@numba.njit()
def forward_it(m,d,ipos1,ipos2,f1,f2,e1,e2):

  for i in range(ipos1.shape[0]):
   d[i]+=m[ipos2[i],ipos1[i]]*f1[i]*f2[i]+\
      m[ipos2[i]+1,ipos1[i]]*f1[i]*e2[i]+\
      m[ipos2[i],ipos1[i]+1]*e1[i]*f2[i]+\
      m[ipos2[i]+1,ipos1[i]+1]*e1[i]*e2[i]
@numba.njit()
def adjoint_it(m,d,ipos1,ipos2,f1,f2,e1,e2):

  for i in range(ipos1.shape[0]):
   m[ipos2[i],ipos1[i]]+=d[i]*f1[i]*f2[i]
   m[ipos2[i]+1,ipos1[i]]+=d[i]*e2[i]*f1[i]
   m[ipos2[i],ipos1[i]+1]+=d[i]*e1[i]*f2[i]
   m[ipos2[i]+1,ipos1[i]+1]+=d[i]*e1[i]*e2[i]


In [ ]:
print(type(vec))
array=vec.get_nd_array()
x,y,z=array[:,0],array[:,1],array[:,2]
xar,yar,zar=array[:,0],array[:,1],array[:,2]

print(x.min(),x.max(),y.min(),y.max())
print((212.8-198.7)/.05,(255.552-234.7)/.05)

In [ ]:
import sep_python.sep_vector as vec_class
model=vec_class.get_sep_vector(ns=[284,418],os=[198.7,234.7],ds=[.05,.05])
data=vec_class.get_sep_vector(z)
oper=linear_interp_2d(model,data,x,y)
oper.dotTest(verbose=True)

In [ ]:
from genericSolver.pyProblem import ProblemL2Linear
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
from sep_plot import Grey
import holoviews as hv
hv.extension('bokeh','matplotlib')

prob=ProblemL2Linear(model,data,oper)
stop=BasicStopper(niter=10000)
solve=LCGsolver(stop)
solve.run(prob)



Check out the ranges of the data

In [ ]:
print(prob.data.min(),prob.data.max(),"data range")
print(prob.model.min(),prob.model.max(),"model range")

The solution blew up. The reason is that the inversion problem is putting very large value next to very small values trying to explain the inconsistency in the data.

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
print(prob.data.min(),prob.data.max())
Grey(prob.model,transp=False,bclip=-250,eclip=-210,invert_yaxis=False)

In [ ]:
c=prob.model.get_nd_array()
c.shape
print(prob.model.get_hyper(),c.shape)

In [ ]:
class igrad2(Operator):

  def __init__(self,mod,dat):

    if not isinstance(mod,FloatVector) or not isinstance(dat,FloatVector):
      raise Exception("model and data must be FloatVectors")
    
    nmod=mod.get_hyper().get_ns()
    ndat=dat.get_hyper().get_ns()
    if len(nmod)!=2 or len(ndat)!=3:
      raise Exception("Unacceptable dimension")
    
    if nmod[0]!=ndat[0] or nmod[1]!=ndat[1] or ndat[2]!=2:
      raise Exception("Model and data size don't work")
    
    super().__init__(mod,dat)

  def forward(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      data.zero()
    d=data.get_nd_array()
    m=model.get_nd_array()

    d[0,:,:-1]+=m[:,1:]-m[:,:-1]
    d[1,:-1,:]+=m[1:,:]-m[:-1,:]

  def adjoint(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      model.zero()

    d=data.get_nd_array()
    m=model.get_nd_array() 

    m[:,1:]+=d[0,:,:-1]
    m[:,:-1]-=d[0,:,:-1]
    m[1:,:]+=d[1,:-1,:]
    m[:-1,:]-=d[1,:-1,:]

In [ ]:
class laplacian_2d(Operator):

  def __init__(self,mod,dat):

    if not isinstance(mod,FloatVector) or not isinstance(dat,FloatVector):
      raise Exception("model and data must be FloatVectors")
    
    nmod=mod.get_hyper().get_ns()
    ndat=dat.get_hyper().get_ns()
    if len(nmod)!=2 or len(ndat)!=2:
      raise Exception("Unacceptable dimension")
    
    if nmod[0]!=ndat[0] or nmod[1]!=ndat[1]:
      raise Exception("Model and data size don't work")
    
    super().__init__(mod,dat)

  def forward(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      data.zero()
    d=data.get_nd_array()
    m=model.get_nd_array()

    d[1:-1,1:-1]+=m[1:-1,1:-1]-.25*(m[0:-2,1:-1]+m[2:,1:-1]+m[1:-1,0:-2]+m[1:-1,2:])

  def adjoint(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      model.zero()

    d=data.get_nd_array()
    m=model.get_nd_array() 

    m[1:-1,1:-1]+=d[1:-1,1:-1]
    m[0:-2,1:-1]-=.25*d[1:-1,1:-1]
    m[2:,1:-1]-=.25*d[1:-1,1:-1]
    m[1:-1,0:-2]-=.25*d[1:-1,1:-1]
    m[1:-1,2:]-=.25*d[1:-1,1:-1]


In [ ]:
from genericSolver.pyProblem import ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
from sep_plot import Grey


ns=model.get_hyper().get_ns()
ns.append(2)
regSpace=vec_class.get_sep_vector(ns=ns)
lap2=laplacian_2d(model,model)
grad2=igrad2(model,regSpace)

eps=1
model.zero()
prob=ProblemL2LinearReg(model,data,oper,epsilon=eps,reg_op=grad2)
stop=BasicStopper(niter=10000)

solve=LCGsolver(stop)
solve.setDefaults(save_obj=True,save_res=True)
solve.run(prob)

In [ ]:
print(prob.data.min(),prob.data.max(),"data range")
print(prob.model.min(),prob.model.max(),"model range")

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
print(prob.data.min(),prob.data.max())
Grey(prob.model,transp=False,bclip=-250,eclip=-210,invert_yaxis=False,width=700,height=450)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
from sep_plot import Graph
res=prob.res.vecs[0].get_nd_array()
res_data=res[:100000].reshape((100,1000))
Graph(res_data,width=1000)

In [ ]:
res=prob.res.vecs[1].get_nd_array()


In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
from sep_plot import Graph
res=prob.res.vecs[1].get_nd_array().ravel()
res2=res[:59000].reshape((59,1000))
Graph(res2,width=1000)

In [ ]:
import holoviews as hv
import sep_python.sep_vector
hv.extension('bokeh','matplotlib')
Grey(prob.res.vecs[1].get_nd_array()[1,:,:],transp=False,invert_yaxis=False,width=700,height=450)

In [ ]:
path=np.vectorize(complex)(xar,yar)




In [ ]:
import math
xdif=np.copy(xar)
ydif=np.copy(yar)
xdif[:-1]=xar[1:]-xar[:-1]
ydif[:-1]=yar[1:]-yar[:-1]
xdif[-1]=xdif[-2]
ydif[-1]=ydif[-2]
ipath=np.zeros(ydif.shape,dtype=np.int32)

@numba.njit()
def calc_angles(xdif,ydif):
  ang=np.copy(xdif)
  for i in range(xdif.shape[0]-1):
    num=xdif[i]*xdif[i+1]*ydif[i]*ydif[i+1]
    den=math.sqrt(xdif[i]*xdif[i]+ydif[i]*ydif[i])*\
      math.sqrt(xdif[i+1]*xdif[i+1]+ydif[i+1]*ydif[i+1])
    ang[i]=math.asin(num/(den+.000001))
  ang[-1]=ang[-2]
  ang=ang*180./math.pi
  return ang

@numba.njit()
def find_segment(xdif,ydif,angle,max_shift,angle_shift,path):
  ipath=0
  path[0]=0
  sq_dist=max_shift*max_shift
  for i in range(1,xar.shape[0]):
    distsq=xdif[i-1]*xdif[i-1]+ydif[i-1]*ydif[i-1]
    if distsq > sq_dist or (distsq >.0001 and angle[i-1]>angle_shift):
       ipath+=1
    path[i]=ipath
  return path

angles=calc_angles(xdif,ydif)
path_label=find_segment(xdif,ydif,angles,.2,10.,ipath)  

def create_list(path,path_label):
  paths=[]
  ibeg=0
  lengths=[]
  for i in range(1,path_label.shape[0]):
    if path_label[i]!=path_label[i-1]:
      loc=path[ibeg:i-1]
      lengths.append((i-ibeg))    
      ibeg=i
      paths.append(loc)
  loc=path[ibeg:]
  paths.append(loc)
  lengths.append(path.shape[0]-ibeg)
  return paths,lengths

segments,lengths=create_list(path,path_label)

In [ ]:
 import holoviews as hv
hv.extension('bokeh','matplotlib')
plots=[Graph(segment) for segment in segments]
hv.Overlay(plots).opts(width=700,height=900)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
from sep_plot import Graph
res=prob.res.vecs[1].get_nd_array().ravel()
res2=path[:1000]
Graph(res2,width=1000)

In [ ]:
class scale(Operator):

  def __init__(self,mod,dat,scale):

    if not isinstance(mod,FloatVector) or not isinstance(dat,FloatVector):
      raise Exception("model and data must be FloatVectors")
    
    if not isinstance(scale,np.ndarray):
      raise Exception("Scale must be nd-array")
    
    nmod=mod.get_hyper().get_ns()
    ndat=dat.get_hyper().get_ns()
    if len(nmod)!=2 or len(ndat)!=2 or len(scale.shape)!=2:
      raise Exception("Unacceptable dimension")
    
    if nmod[0]!=ndat[0] or nmod[1]!=ndat[1] or ndat[0]!=scale.shape[0]\
      or ndat[1]!=scale.shape[1]:
      raise Exception("Model,data, shape sizes don't work")
      self._scale=scale
    
    super().__init__(mod,dat)

  def forward(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      data.zero()
    d=data.get_nd_array()
    m=model.get_nd_array()

    d+=self._scale*m

  def adjoint(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      model.zero()

    d=data.get_nd_array()
    m=model.get_nd_array() 

    m+=scale._scale*d

In [ ]:
import scipy.signal
def median_filter(lengths,data):
  data_out=data.clone()
  din=data.get_nd_array()
  dout=data_out.get_nd_array()
  ibeg=0
  for length in lengths:
    dout[ibeg:ibeg+length]=scipy.signal.medfilt(din[ibeg:ibeg+length],9)
    ibeg+=length
  return data_out

d_median=median_filter(lengths,data)

In [ ]:
from genericSolver.pyProblem import ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
from sep_plot import Grey


ns=model.get_hyper().get_ns()
ns.append(2)
regSpace=vec_class.get_sep_vector(ns=ns)
lap2=laplacian_2d(model,model)
grad2=igrad2(model,regSpace)

eps=1
model.zero()
prob_med=ProblemL2LinearReg(model,d_median,oper,epsilon=eps,reg_op=grad2)
stop=BasicStopper(niter=10000)

solve=LCGsolver(stop)
solve.setDefaults(save_obj=True,save_res=True)
solve.run(prob_med)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
print(prob_med.data.min(),prob_med.data.max())
Grey(prob_med.model,transp=False,bclip=-250,eclip=-210,invert_yaxis=False,width=700,height=450)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
from sep_plot import Graph
res=prob_med.res.vecs[0].get_nd_array()
res_m=res[:100000].reshape((100,1000))
hv.Layout(Graph(res_m,width=1000)+Graph(res_data,width=1000)).cols(1)


In [ ]:
import scipy.signal
def median_filter(lengths,data):
  data_out=data.clone()
  din=data.get_nd_array()
  dout=data_out.get_nd_array()
  ibeg=0
  for length in lengths:
    if length >5:
      dtmp=np.zeros((length+4,))
      dtmp[2:-2]=dout[ibeg:ibeg+length]
      dtmp[1]=dtmp[3]
      dtmp[0]=dtmp[4]
      dtmp[-1]=dtmp[-3]
      dtmp[-2]=dtmp[-4]
      dout[ibeg:ibeg+length]=scipy.signal.medfilt(dtmp,5)[2:-2]
    else:
      for i in range(ibeg,ibeg+length):
        if dout[i]==0:
          dout[i]=dout[i-1]
    ibeg+=length
  return data_out

d_median=median_filter(lengths,data)

In [ ]:
from genericSolver.pyProblem import ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
from sep_plot import Grey


ns=model.get_hyper().get_ns()
ns.append(2)
regSpace=vec_class.get_sep_vector(ns=ns)
lap2=laplacian_2d(model,model)
grad2=igrad2(model,regSpace)

eps=1
model.zero()
prob_med2=ProblemL2LinearReg(model,d_median,oper,epsilon=eps,reg_op=grad2)
stop=BasicStopper(niter=10000)

solve=LCGsolver(stop)
solve.setDefaults(save_obj=True,save_res=True)
solve.run(prob_med2)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
from sep_plot import Graph
res=prob_med2.res.vecs[0].get_nd_array()
res_m=res[:100000].reshape((100,1000))
hv.Layout(Graph(res_m,width=1000)+Graph(res_data,width=1000)).cols(1)


In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
from sep_plot import Graph
res=prob_med2.res.vecs[0].get_nd_array()
res_m=res[:100000].reshape((100,1000))
Graph(res_m,width=1000)

In [ ]:
print(res_m[77,750:760].min())
print(res[77750:77760],path_label[77750:77760],data.get_nd_array()[77750:77760])

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
print(prob_med.data.min(),prob_med.data.max())
Grey(prob_med.model,transp=False,bclip=-250,eclip=-210,invert_yaxis=False,width=700,height=450)

In [ ]:
eps_vals=[10.**v for v in np.linspace(-3,3,25)]
objs=np.array([],dtype=np.complex64)
for eps in eps_vals:
  print(eps)
  model.zero()
  solve=LCGsolver(stop)
  prob_med2=ProblemL2LinearReg(model,d_median,oper,epsilon=eps,reg_op=grad2)
  solve.setDefaults(save_obj=True)
  solve.run(prob_med2)
  objs=np.append(objs,complex(eps,solve.obj[-1]))



In [ ]:
import holoviews as hv
print(objs)
hv.extension('bokeh','matplotlib')
Graph(objs,logx=True)

In [ ]:
lin=np.linspace(-1,.5,25)
objs=np.array([],dtype=np.complex64)
for e in lin:
  print(e,10**e)
  model.zero()
  solve=LCGsolver(stop)
  prob_med2=ProblemL2LinearReg(model,d_median,oper,epsilon=10**e,reg_op=grad2)
  solve.setDefaults(save_obj=True)
  solve.run(prob_med2)
  objs=np.append(objs,complex(e,solve.obj[-1]))

In [ ]:
import holoviews as hv
print(objs)
hv.extension('bokeh','matplotlib')
Graph(objs,logx=True)

In [ ]:
class i_op(Operator):

  def __init__(self,model,data):

    if not model.check_same(data):
      raise Exception("Domain and range must be same")
  
    super().__init__(model,data)
  
  def forward(self,add,model,data):
    if not add:
      data.zero()
    
    data.scale_add(model)
  
  def adjoint(self,add,model,data):

    if not add:
      model.zero()

    model.scale_add(data)
  

class fake_op(Operator):

  def __init__(self,model,ipath,op,sc):

    path=sep_python.sep_vector.get_sep_vector(ns=[max(ipath)])
    print(max(ipath),ipath.max(),model.get_nd_array().shape,"SSS")
    mod=superVector(model,path)
    self._iop=i_op(path,path)
    self._op=op
    self._sc=sc
    self._tmp=path.clone()
    data=superVector(op.range,path)

    super().__init__(mod,data)

  def forward(self,add,model,data):

    self.checkDomainRange(model,data)
    self._op.forward(add,model.vecs[0],data.vecs[0])
    self._tmp.scale_add(model.vecs[1],sc1=0,sc2=self._sc)
    self._iop.forward(add,self._tmp,data.vecs[1])
  
  def adjoint(self,add,model,data):
    self.checkDomainRange(model,data)
    self._op.adjoint(add,model.vecs[0],data.vecs[0])
    self._tmp.scale_add(data.vecs[1],sc1=0,sc2=self._sc)
    self._iop.adjoint(add,model.vecs[1],self._tmp)

In [ ]:
from genericSolver.pyVector import superVector

class linear_interp_2d_plus(Operator):
  def __init__(self,model:superVector,data:FloatVector,x,y,ipath):
    if not isinstance(model,superVector) or not isinstance(data,FloatVector):
      raise Exception("wrong input")
    
    axes=model.vecs[0].get_hyper().axes
    if len(axes)!=2:
      raise Exception("expecting model to be 2-D")

    self._f1=(x-axes[0].o)/axes[0].d
    self._f2=(y-axes[1].o)/axes[1].d
    print(type(x),type(y),"XY")
    self._ipos2=np.int_(self._f2)
    self._ipos1=np.int_(self._f1)
    self._f1-=self._ipos1
    self._f2-=self._ipos2
    self._e1=1.-self._f1
    self._e2=1.-self._f2
    self._ipath=ipath
    find_outside(axes[0].n,axes[1].n,self._ipos1,self._ipos2,self._f1,\
        self._f2,self._e1,self._e2)
    super().__init__(model,data)

  def forward(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      data.zero()
    d=data.get_nd_array()
    m=model.vecs[0].get_nd_array()
    p=model.vecs[1].get_nd_array()
    forward_it2(m,p,d,self._ipath,self._ipos1,self._ipos2,self._f1,self._f2,self._e1,self._e2)
    
  def adjoint(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      model.zero()
    m=model.vecs[0].get_nd_array()
    p=model.vecs[1].get_nd_array()
    d=data.get_nd_array()
    adjoint_it2(m,p,d,self._ipath,self._ipos1,self._ipos2,self._f1,self._f2,self._e1,self._e2)

  

@numba.njit()
def forward_it2(m,p,d,ipath,ipos1,ipos2,f1,f2,e1,e2):

  for i in range(ipos1.shape[0]):
   d[i]+=m[ipos2[i],ipos1[i]]*f1[i]*f2[i]+\
      m[ipos2[i]+1,ipos1[i]]*f1[i]*e2[i]+\
      m[ipos2[i],ipos1[i]+1]*e1[i]*f2[i]+\
      m[ipos2[i]+1,ipos1[i]+1]*e1[i]*e2[i]+p[ipath[i]]
@numba.njit()
def adjoint_it2(m,p,d,ipath,ipos1,ipos2,f1,f2,e1,e2):

  for i in range(ipos1.shape[0]):
   m[ipos2[i],ipos1[i]]+=d[i]*f1[i]*f2[i]
   m[ipos2[i]+1,ipos1[i]]+=d[i]*e2[i]*f1[i]
   m[ipos2[i],ipos1[i]+1]+=d[i]*e1[i]*f2[i]
   m[ipos2[i]+1,ipos1[i]+1]+=d[i]*e1[i]*e2[i]
   p[ipath[i]]+=d[i]

In [ ]:
a1=new_model.vecs[0]
a2=new_model.vecs[1]
print(a1.get_nd_array().shape,a2.get_nd_array().shape)

In [ ]:
eps=1
model.zero()
new_reg=fake_op(model,ipath,grad2,1.)
new_model=new_reg.getDomain().clone()
op_new=linear_interp_2d_plus(new_model,data,x,y,ipath)
prob_med2=ProblemL2LinearReg(new_model,d_median,op_new,epsilon=eps,reg_op=new_reg)
stop=BasicStopper(niter=10000)

solve=LCGsolver(stop)
solve.setDefaults(save_obj=True,save_res=True)
solve.run(prob_med2)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
print(prob_med2.data.min(),prob_med.data.max())
Grey(prob_med2.model.vecs[0],transp=False,bclip=-250,eclip=-210,invert_yaxis=False,width=700,height=450)

In [ ]:
import holoviews as hv
Graph(prob_med2.model.vecs[1],width=600)

In [ ]:
res=prob_med2.res.vecs[0].get_nd_array()
res_m=res[:100000].reshape((100,1000))
Graph(res_m,width=1000)

In [ ]:
import matplotlib.pyplot as plt
res.shape

In [ ]:
arr=np.histogram(res,bins=np.linspace(-2,2,500))
Graph(sep_python.sep_vector.get_sep_vector(arr[0].astype("float32")))